In [18]:
import astropy.units as u
import astropy.constants as C
import importlib
import units.smm as smu
import logging
importlib.reload(smu)

<module 'units.smm' from 'e:\\Code\\autism\\ss13\\supermattercore\\units\\smm.py'>

In [19]:
smu.PumpLogger.setLevel(logging.DEBUG)
%config Application.logging_config = {'loggers': {'ct.sm': {'propagate': True, 'level': 'DEBUG', 'handlers': ['console']}}}

In [20]:
core_room = smu.GasMixture(smu.T_MINIMAL, 15*smu.CELL_VOLUME, {})
smcore = smu.Supermatter(core_room, smu.POWERED_EMITTER_DMG*8 / smu.SM_EER_EMIT)

In [21]:
canist_ref = smu.GasMixture.pure('N2', 293.15*u.K, 1000*u.L, 90*smu.atm)
canist_ref

GasMixture(T=<Quantity 293.15 K>, V=<Quantity 1000. l>, nus={'N2': <Quantity 3743.41692471 mol>})

In [22]:
cooled   = smu.PipeNetwork({'cooled': smu.GasMixture(smu.T_MINIMAL, 70*u.L)})
cooling  = smu.PipeNetwork({'cooling': smu.GasMixture(smu.T_MINIMAL, 15680*u.L)})
warm     = smu.PipeNetwork({'warm': smu.GasMixture(smu.T_MINIMAL, 1890*u.L)})
scalding = smu.PipeNetwork({'scalding': smu.GasMixture(smu.T_MINIMAL, 315*u.L)})
# core_    = smu.PipeNetwork({'room': core_room})

In [23]:
po1 = smu.OutletInjector('PO1', P_max=30*u.kW, target_V=700*u.L) # Outlet Injector
pi1 = smu.VentPump('PI1', V_out=700*u.L, ex_P = 100*u.kPa) # , V_in=0*u.L, V_out=0.2*u.m**3) # Vent Pump
hp1 = smu.Pump('HP1', P_max=45*u.kW)
pc_hot =  smu.Pump('P[C]hot') # C2-P2 on pic
pc_cold = smu.Pump('P[C]cold') # C5-P6 on pic
thot =  smu.Turbine('T(hot)1')
tcold = smu.Turbine('T(cold)2')
teg = smu.TEG(thot, tcold)
he = smu.HeatExchanger(191)


In [24]:
chot = smu.PipeNetwork({'':canist_ref*1})
ccold = smu.PipeNetwork({'':canist_ref*1})
pc_hot.connect (chot, warm)
pc_cold.connect(ccold, cooling)
hp1.connect(cooling, cooled)
pi1.connect(core_room, scalding)
po1.connect(warm, core_room)
thot.connect(scalding, warm)
tcold.connect(cooled, cooling)
# warm.V += 2*u.m**3 # Filters
warm.add('filter1', 1*u.m**3)
warm.add('filter2', 1*u.m**3)
warm.add('nopump', .2*u.m**3) # empty for waste or whatever

probably worth it to make a parser of neb debug logs to match with own debug logs

current error for cold loop on its own is sub 1e-6. probably good enough.

In [43]:
for _ in range(10):
    ([x.pump() for x in [pc_hot, pi1, po1]])
    (teg.process())
    [x.equalize() for x in [chot,  warm, scalding]]
core_room

GasMixture(T=<Quantity 293.15 K>, V=<Quantity 37500. l>, nus={'N2': <Quantity 1699.48861633 mol>})

In [44]:
scalding.gases

{'scalding': GasMixture(T=<Quantity 2.7 K>, V=<Quantity 315. l>, nus={}),
 'PI1': GasMixture(T=<Quantity 2.7 K>, V=<Quantity 700. l>, nus={}),
 'T(hot)1': GasMixture(T=<Quantity 2.7 K>, V=<Quantity 400. l>, nus={})}

In [46]:
%config Application.log_level = 'INFO'
for _ in range(1):
    print([x.pump() for x in [pc_hot, pi1, po1]])
    print(teg.process())
    [x.equalize() for x in [chot,  warm, scalding]]
%config Application.log_level = 30
print(pc_hot.inlet.gases[''].p.to(u.kPa), pc_cold.inlet.gases[''].p.to(u.kPa))
print(core_room.p.to(u.kPa), warm.gases['warm'].p.to(u.kPa), scalding.gases['scalding'].p.to(u.kPa))

[ct.sm.Pump] P[C]hot: Source entropy 171.609 J / (K mol) -> Sink entropy 175.601 J / (K mol)
[ct.sm.Pump] P[C]hot: Specific entropy change 3.99219 J / (K mol)
[ct.sm.Pump] P[C]hot: Specific power 0 J / mol
[ct.sm.Pump] P[C]hot: Transferred 83.9698 mol in fact
[ct.sm.Pump] P[C]hot: Sink stat = 293.15 K, 51.8681 mol, 631.774 kPa
[ct.sm.Pump] P[C]hot Merged-sink stat: 293.15 K, 135.8 mol, 1654.56 kPa
[ct.sm.Pump] PI1 target dP=22.97430600859011 kPa
[ct.sm.Pump] PI1: Source entropy 189.186 J / (K mol) -> Sink entropy 228.872 J / (K mol)
[ct.sm.Pump] PI1: Specific entropy change 39.6857 J / (K mol)
[ct.sm.Pump] PI1: Specific power 0 J / mol
[ct.sm.Pump] PI1: Transferred 1.32975 mol in fact
[ct.sm.Pump] PI1: Sink stat = 293.15 K, 0.29785 mol, 1.03655 kPa
[ct.sm.Pump] PI1 Merged-sink stat: 293.15 K, 1.628 mol, 5.66423 kPa
[ct.sm.Pump] PO1: Source entropy 175.601 J / (K mol) -> Sink entropy 189.192 J / (K mol)
[ct.sm.Pump] PO1: Specific entropy change 13.5911 J / (K mol)
[ct.sm.Pump] PO1: Spec

[<Quantity 0. J>, <Quantity 0. J>, <Quantity 0. J>]
0.0 J
852.3204784538345 kPa 7599.375 kPa
134.68103102219777 kPa 585.9770074146468 kPa 3.3258615130490004 kPa


In [47]:
%config Application.log_level = 'INFO'
(pc_cold.pump())
ccold.equalize()
cooling.equalize()
(pc_cold.pump())
for _ in range(10):
    # smu.SmLogger.info(he.approx_cool(cooling.gases['cooling']))
    he.cool(cooling.gases['cooling'])
    ccold.equalize()
    cooling.equalize()
    (pc_cold.pump())
    # cooling.equalize()
    # he2.cool(cooling.gases['cooling'])
    # cooling.equalize()
%config Application.log_level = 30

[ct.sm.Pump] P[C]cold: Source entropy 155.129 J / (K mol) -> Sink entropy 150000 J / (K mol)
[ct.sm.Pump] P[C]cold: Specific entropy change 149845 J / (K mol)
[ct.sm.Pump] P[C]cold: Specific power 0 J / mol
[ct.sm.Pump] P[C]cold: Transferred 623.903 mol in fact


[ct.sm.Pump] P[C]cold: Sink stat = 2.7 K, 0 mol, 0 kPa
[ct.sm.Pump] P[C]cold Merged-sink stat: 293.15 K, 623.9 mol, 7599.37 kPa
[ct.sm.Pump] P[C]cold: Source entropy 156.608 J / (K mol) -> Sink entropy 191.475 J / (K mol)
[ct.sm.Pump] P[C]cold: Specific entropy change 34.8669 J / (K mol)
[ct.sm.Pump] P[C]cold: Specific power 0 J / mol
[ct.sm.Pump] P[C]cold: Transferred 519.919 mol in fact
[ct.sm.Pump] P[C]cold: Sink stat = 293.15 K, 7.66465 mol, 93.3584 kPa
[ct.sm.Pump] P[C]cold Merged-sink stat: 293.15 K, 527.6 mol, 6426.17 kPa
[ct.sm] last -9.18464423578679 J; Per 191 pipes 293.15 K -> 293.00 K (0.1461 K loss)
[ct.sm.Pump] P[C]cold: Source entropy 158.094 J / (K mol) -> Sink entropy 186.441 J / (K mol)
[ct.sm.Pump] P[C]cold: Specific entropy change 28.3474 J / (K mol)
[ct.sm.Pump] P[C]cold: Specific power 0 J / mol
[ct.sm.Pump] P[C]cold: Transferred 433.266 mol in fact
[ct.sm.Pump] P[C]cold: Sink stat = 293.07 K, 14.0519 mol, 171.112 kPa
[ct.sm.Pump] P[C]cold Merged-sink stat: 293.07

In [35]:
%config Application.log_level = 'INFO'
for _ in range(10):
    # smu.SmLogger.info(he.approx_cool(cooling.gases['cooling']))
    he.cool(cooling.gases['cooling'])
    ccold.equalize()
    cooling.equalize()
    (pc_cold.pump())
    # cooling.equalize()
    # he2.cool(cooling.gases['cooling'])
    # cooling.equalize()
%config Application.log_level = 30

[ct.sm] last -51.46478810844523 J; Per 191 pipes 288.31 K -> 288.17 K (0.1365 K loss)
[ct.sm.Pump] P[C]cold: Source entropy 218.548 J / (K mol) -> Sink entropy 176.65 J / (K mol)
[ct.sm.Pump] P[C]cold: Specific entropy change -41.8975 J / (K mol)
[ct.sm.Pump] P[C]cold: Specific power 4829.6 J / mol
[ct.sm.Pump] P[C]cold: Transferred 0.294784 mol in fact
[ct.sm.Pump] P[C]cold: Sink stat = 288.18 K, 45.9662 mol, 550.393 kPa
[ct.sm.Pump] P[C]cold: Source entropy 218.548 J / (K mol) -> Sink entropy 176.65 J / (K mol)
[ct.sm.Pump] P[C]cold: Specific entropy change -41.8975 J / (K mol)
[ct.sm.Pump] P[C]cold: Specific power 4829.6 J / mol
[ct.sm.Pump] P[C]cold: Transferred 0.294784 mol in fact
[ct.sm.Pump] P[C]cold: Sink stat = 288.18 K, 45.9662 mol, 550.393 kPa
[ct.sm.Pump] P[C]cold Merged-sink stat: 288.2113 K, 46.26 mol, 553.983 kPa
[ct.sm] last -51.37434032492347 J; Per 191 pipes 288.18 K -> 288.04 K (0.1363 K loss)
[ct.sm.Pump] P[C]cold: Source entropy 220.063 J / (K mol) -> Sink entropy

technically prev level

In [274]:
%config Application.log_level = 'INFO'
for _ in range(1):
    print([x.pump() for x in [pc_hot, po1, pi1]])
    print(teg.process())
    # print(he.cool(cooling.gases['cooling']).T)
    # ccold, cooled, cooling,
    [x.equalize() for x in [chot,  warm, scalding]]
%config Application.log_level = 30
print(pc_hot.inlet.gases[''].p.to(u.kPa), pc_cold.inlet.gases[''].p.to(u.kPa))
print(core_room.p.to(u.kPa), warm.gases['warm'].p.to(u.kPa), scalding.gases['scalding'].p.to(u.kPa))

[ct.sm.Pump] P[C]hot: Source entropy 164.077 J / (K mol) -> Sink entropy 170.505 J / (K mol)
[ct.sm.Pump] P[C]hot: Specific entropy change 6.42846 J / (K mol)
[ct.sm.Pump] P[C]hot: Specific power 0 J / mol
[ct.sm.Pump] P[C]hot: Transferred 208.944 mol in fact
[ct.sm.Pump] P[C]hot: Sink stat = 293.15 K, 95.9443 mol, 1168.64 kPa
[ct.sm.Pump] Merged-sink stat: 293.15 K, 304.9 mol, 3713.65 kPa
[ct.sm.Pump] PO1: Source entropy 170.505 J / (K mol) -> Sink entropy 150000 J / (K mol)


[ct.sm.Pump] PO1: Specific entropy change 149829 J / (K mol)
[ct.sm.Pump] PO1: Specific power 0 J / mol
[ct.sm.Pump] PO1: Transferred 0 mol in fact
[ct.sm.Pump] PO1: Sink stat = 2.7 K, 0 mol, 0 kPa
[ct.sm] T(cold)2: merged: inP=0 Pa, outP=5.4717e+05 Pa
[ct.sm] T(hot)1: merged: inP=0 Pa, outP=1.1686e+06 Pa


[<Quantity 0. J>, <Quantity 0. J>, <Quantity 0. W>]
0.0 J
2120.846092946246 kPa 0.27965854715851235 kPa
0.0 kPa 547.1681974842829 kPa 1266.7118860239893 kPa 0.0 kPa


together pumps

In [30]:
chot.gases[''] = canist_ref * 1
ccold.gases[''] = canist_ref * 1

In [56]:
for _ in range(15):
    ([x.pump() for x in [pc_hot, pc_cold, po1, pi1, hp1]])
    (teg.process())
    smcore.process()
    if i>1 : he.cool(cooling.gases['cooling'])
    [x.equalize() for x in [chot,  warm, scalding, ccold, cooling, cooled]]
    if cT0 is not None:
        print('Core dT', (core_room.T - cT0))
        cT0 = None
    (pc_cold.pump())

print(f'Canisters: {pc_hot.inlet.gases[""].p.to(u.kPa):.3g} hot, {pc_cold.inlet.gases[""].p.to(u.kPa):.3g} cold')
print(f'Core: {smcore.eer:.3f}, {core_room.T:.2f}, {core_room.p.to(u.kPa):.2f}')
print('warm',    warm.gases['warm'].p.to(u.kPa), warm.gases['warm'].T,       '')
print('cooling', cooling.gases['cooling'].p.to(u.kPa), cooling.gases['cooling'].T, '')
print('scalding',scalding.gases['scalding'].p.to(u.kPa), scalding.gases['scalding'].T)
print(cooled.gases['cooled'].p.to(u.kPa))
cT0 = core_room.T

Core dT -0.15667315126302128 K
Canisters: 0.112 kPa hot, 0.104 kPa cold
Core: 99.696 EER, 2650.89 K, 6025.85 kPa
warm 3638.273988585842 kPa 1431.3612617585973 K 
cooling 3025.6115073830306 kPa 559.3343982114376 K 
scalding 5418.442898313371 kPa 2647.7938893836035 K
4393.945905589115 kPa


In [57]:
{k: v/core_room.Nu for k,v in core_room.nus.items()}

{'O2': <Quantity 0.05351593>,
 'Ph': <Quantity 0.02564187>,
 'N2': <Quantity 0.9208422>}

In [58]:
teg.process()

<Quantity 863013.9503749 J>

In [53]:
core_room.T

<Quantity 2657.93654546 K>